In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup
import re
import json

In [4]:
with open('Sky_sports_result.json', 'rb') as outfile:
    urls = json.load(outfile)

In [5]:
headers = {'Access-Control-Allow-Origin': '*',
           'Access-Control-Allow-Methods': 'GET',
           'Access-Control-Allow-Headers': 'Content-Type',
           'Access-Control-Max-Age': '3600',
           'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
          }

In [7]:
epl_match_result_data = {}
for url in urls:
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')

    __#Extract Date time__
    date_time = soup.find(class_='sdc-site-match-header__detail-time').getText()

    date = ""
    time = ""
    weekday=""
    match = re.match(r"([0-9:a-z]+)(,\s)([a-z]+)(\s)([0-9a-z\s]+)(.)",date_time,re.I)
    if match:
        res_lst = match.groups()
        time = res_lst[0].strip()
        weekday = res_lst[2].strip()
        date = res_lst[4].strip()
    
    __#Get Team names__
    teams= soup.findAll(class_="sdc-site-match-header__team-name-block-target")
    home = teams[0].getText()
    away = teams[1].getText()
    
    __#Get Scores__
    scores = soup.findAll(class_="sdc-site-match-header__team-score-block")
    home_score = scores[0].getText()
    away_score = scores[1].getText()
    
    __#Get headline__
    short_summary = soup.find(class_="sdc-article-header__long-title").getText()
    
    
    __#Get venue __
    venue=""
    try:
        venue = soup.find(class_="sdc-site-match-header__detail-venue sdc-site-match-header__detail-venue--with-seperator").getText()
    except:
        __#print("Not found for"+url)__
        venue = soup.find(class_="sdc-site-match-header__detail-venue").getText()
    finally:
        print(venue)
    __#Get Gist__
    gist = soup.find(class_="sdc-article-body sdc-article-body--lead").findAll('p')[0].getText()
    detail = soup.find(class_="sdc-article-body sdc-article-body--lead").findAll('p')[1].getText()
    
    id = date_time+"_"+home+"_"+away
    epl_match_result_data[id] = {'home':home,'away':away, 'home_score': home_score,'away_score':away_score,'news':short_summary,'response': gist,'detail':detail,'time':time,'weekday':weekday,'venue':venue,'date':date}
    __#print(epl_match_result_data[id])__

Emirates Stadium.
Turf Moor.
Stamford Bridge.
Selhurst Park.
Goodison Park.
The King Power Stadium.
Etihad Stadium.
St James' Park, Newcastle.
St. Mary's Stadium.
London Stadium.
Old Trafford.
Anfield.
Vicarage Road.
Villa Park.
Amex Stadium.
Bramall Lane.
Molineux.
Vitality Stadium.
Tottenham Hotspur Stadium.
Carrow Road.
London Stadium.
Goodison Park.
The King Power Stadium.
Selhurst Park.
St. Mary's Stadium.
Turf Moor.
Etihad Stadium.
St James' Park, Newcastle.
Emirates Stadium.
Stamford Bridge.
Old Trafford.
Molineux.
Villa Park.
Tottenham Hotspur Stadium.
Vitality Stadium.
Carrow Road.
Vicarage Road.
Anfield.
Bramall Lane.
Amex Stadium.
Vitality Stadium.
Goodison Park.
Villa Park.
Etihad Stadium.
Bramall Lane.
London Stadium.
Amex Stadium.
Selhurst Park.
Vicarage Road.
Emirates Stadium.
Tottenham Hotspur Stadium.
Turf Moor.
St James' Park, Newcastle.
Anfield.
St. Mary's Stadium.
Carrow Road.
The King Power Stadium.
Old Trafford.
Molineux.
Stamford Bridge.
Bramall Lane.
Etihad Stad

In [8]:
with open('EPL_match_caption_new.json', 'w') as outfile:
    json.dump(epl_match_result_data, outfile,indent=2)

In [9]:
training_file= open("training_final.txt","w",encoding='utf-8')
for k,items in epl_match_result_data.items():
    result = ""
    #print(items)
    if int(items['home_score']) > int(items['away_score']):
        result="win"
    elif int(items['home_score']) < int(items['away_score']):
        result="lose"
    else:
        result="draw"
    line = "<match_result> @ <inform> ( <subject>="+items['home']+";"+"<home>="+items['home']+";"+"<home_score>="+items['home_score']+";"+"<away>="+items['away']+";"+"<away_score>="+items['away_score']+";"+"<result>="+result+";"+"<venue>="+items['venue']+";"+"<weekday>="+items['weekday']+";"+"<news>="+items['news']+")"+" & "+items['response']+"\n"
    #print(line)
    training_file.write(line)
training_file.close()